In [258]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import *
import time
import pandas as pd
from tqdm import tqdm
import concurrent.futures
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [259]:
# Setting up the web-browser

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)

action = ActionChains(driver)

In [248]:
# Homepage url for NFT Collections

homepage_url = 'https://opensea.io/collection/boredapeyachtclub/activity?search[eventTypes][0]=OFFER_ENTERED'

In [260]:
df = pd.DataFrame(columns=['nft_id','price_crypto','price_usd','quantity','offer_from_name','offer_from_link','time_uploaded'])

In [261]:
df

,nft_id,price_crypto,price_usd,quantity,offer_from_name,offer_from_link,time_uploaded


In [262]:
# Getting the webpage till bottom
def get_data(url):

    
    driver.maximize_window()
    n = 0
    driver.implicitly_wait(3)
    # Wait for the page to load
    wait = WebDriverWait(driver, 4)

    height = driver.execute_script('return document.body.scrollHeight;')
    all_items= driver.find_elements(by=By.XPATH,value="//*[@role='listitem']")
    hover_button= driver.find_elements(by=By.XPATH,value="//*[@class='sc-29427738-0 sc-f95e0497-0 ikrGyo frmfzd']") #sc-29427738-0 sc-f95e0497-0 ikrGyo frmfzd      

    for i in range(len(all_items)):
        items_list = all_items[i].text.split('\n')
        item_link = all_items[i].find_element(by=By.XPATH,value="//*[@class='sc-1f719d57-0 hoTuIF sc-29427738-0 ikrGyo AccountLink--ellipsis-overflow']").get_attribute('href')
        action.move_to_element(hover_button[i]).click().perform()
        abs_time = driver.find_element(by=By.XPATH,value="//*[@class='sc-29427738-0 sc-f95e0497-0 ikrGyo frmfzd']/div").text
        
        #Enter the extracted data into dataframe
        df.loc[i] = [items_list[1],items_list[3],items_list[4],items_list[6],item_link,items_list[7],abs_time]
        # print("************************")

    
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(3)

    all_items= driver.find_elements(by=By.XPATH,value="//*[@role='sc-29427738-0 sc-4e15afd9-0 dVNeWL kcFcYa']")
    hover_button= driver.find_elements(by=By.XPATH,value="//*[@class='sc-29427738-0 sc-f95e0497-0 ikrGyo frmfzd']")
    print("Hover but: ",hover_button)
    
    try:
        pos = len(df)
        for k in range(len(all_items)):
            items_list = all_items[k].text.split('\n')
            item_link = all_items[k].find_element(by=By.XPATH,value="//*[@class='sc-1f719d57-0 hoTuIF sc-29427738-0 ikrGyo AccountLink--ellipsis-overflow']").get_attribute('href')
            action.move_to_element(hover_button[k]).click().perform()
            abs_time = driver.find_element(by=By.XPATH,value="//*[@class='sc-29427738-0 sc-f95e0497-0 ikrGyo frmfzd']/div").text
            
            #Enter the extracted data into dataframe
            df.loc[pos+k] = [items_list[1],items_list[3],items_list[4],items_list[6],item_link,items_list[7],abs_time]
       

    except StaleElementReferenceException:
        pass
    
    

In [265]:
driver.get(homepage_url)

In [266]:
get_data(homepage_url)

Hover but:  [<selenium.webdriver.remote.webelement.WebElement (session="0e3b255a5546e6c85f50205d8be5b227", element="819dfb9f-8712-4e4e-8e42-db4e91cbb083")>, <selenium.webdriver.remote.webelement.WebElement (session="0e3b255a5546e6c85f50205d8be5b227", element="84d3e353-c1d3-4569-9b94-f5fef0c160fb")>, <selenium.webdriver.remote.webelement.WebElement (session="0e3b255a5546e6c85f50205d8be5b227", element="3f984511-daef-415c-b539-49e6bcce3d5d")>, <selenium.webdriver.remote.webelement.WebElement (session="0e3b255a5546e6c85f50205d8be5b227", element="1ec9b021-9c85-44ee-8e94-ef9def389f7c")>, <selenium.webdriver.remote.webelement.WebElement (session="0e3b255a5546e6c85f50205d8be5b227", element="fb4df01a-6972-4760-92ab-0010897178e5")>, <selenium.webdriver.remote.webelement.WebElement (session="0e3b255a5546e6c85f50205d8be5b227", element="97132cb3-5c23-47ae-8c34-fc0f82ad0006")>, <selenium.webdriver.remote.webelement.WebElement (session="0e3b255a5546e6c85f50205d8be5b227", element="09947687-dfd8-443b-a

In [267]:
df

,nft_id,price_crypto,price_usd,quantity,offer_from_name,offer_from_link,time_uploaded
0,2665,61.0616 WETH,"$111,274.39",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:38 AM"
1,417,62.0033 WETH,"$112,990.49",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:37 AM"
2,9734,62.0033 WETH,"$112,990.49",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:37 AM"
3,8834,61.0616 WETH,"$111,274.39",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:37 AM"
4,235,62.5033 WETH,"$113,901.64",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:37 AM"
5,2458,61.0616 WETH,"$111,274.39",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:37 AM"
6,520,62.0033 WETH,"$112,990.49",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:38 AM"
7,9435,61.0616 WETH,"$111,274.39",1,https://opensea.io/NFTButlerGPT,NFTButlerGPT,"March 30, 2023 at 10:38 AM"
8,215,60.020 WETH,"$109,376.25",1,https://opensea.io/NFTButlerGPT,icebreaker-CheaperLin...,"March 30, 2023 at 10:43 AM"


In [268]:
df.to_csv('data_opensea.csv')